# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**.

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Просмотр  данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')
import time
import collections, numpy

df=pd.read_csv('/datasets/users_behavior.csv')
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [2]:
df.info()
print()
print('Количество дубликатов:', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB

Количество дубликатов: 0


Пропусков и дубликтов нет. В таблице есть данные об истории используемых параметров тарифа: звонки, смс и интернет. Это будут признаки для моделей. А также данные о том являтся ли тариф Ulta-1 или Smart-0 - это будет целевой признак.

## Разделение данных на выборки

Сначала разобъем данные на признаки и целевой признак

In [3]:
targets = df['is_ultra'] # столбец с целевым признаком
features = df.drop(['is_ultra'], axis=1) # таблица с признаками. 

Для обучения и проверки модели разобъем данные на 3 выборки: обучающая, валидационная и тестовая в пропорции 60:20:20. 

Для начала выделим 20% для тестовой выборки. 80% попадут в обучающую

In [4]:
features_train, features_test, targets_train, targets_test = train_test_split(
        features,targets,stratify=targets, test_size=0.2, random_state=12345)

Теперь выделим 20% для валидационной выборки  от обучаюшей выборки

In [5]:
features_train, features_valid, targets_train, targets_valid = train_test_split(
        features_train, targets_train, stratify=targets_train,test_size=0.25, random_state=12345)

Получили 3 выборки. 60% - обучающая выборка, 20% - валидационная и 20%- тестовая

## Исследование моделей

Модель **Решающее дерево**

Определим по валидационной выборке с какой глубиной дерева получается наилучший результат с долей правильных ответов. 

In [6]:
best_model= None
best_result=0
for depth in range (1,10):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, targets_train)
    result = model_tree.score(features_valid, targets_valid)
    if result>best_result:
        best_model= model_tree
        best_result= result
print(best_model)
print('Accuracy:', best_result)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')
Accuracy: 0.8164852255054432


С max_depth=5 получается доля правильных ответов 0.81, что больше чем 0.75

Модель **Случайный лес**

Определим по валидационной выборке с какими параметрами получается наилучший результат с долей правильных ответов.
Проверять будем гиперпараметры max_depth и n_estimators

In [7]:
best_model= None
best_result=0
for depth in range (1,10):
    for est in range(1,20):
        model_forest = RandomForestClassifier(random_state=12345, max_depth=depth,  n_estimators=est)
        model_forest.fit(features_train, targets_train)
        result = model_forest.score(features_valid, targets_valid)
        if result>best_result:
            best_model= model_forest
            best_result= result
print(best_model)
print(best_result)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)
0.8320373250388803


С max_depth=8 и n_estimators=10 получается доля правильных ответов 0.83, что больше чем 0.75

Модель **Логистическая регрессия**

Модель имеет мало парамеров и не склонна к переобучению.Проверим ее расчет на валидационной выборке

In [8]:
model_reg= LogisticRegression(random_state=12345)
model_reg.fit(features_train, targets_train)
result=model_reg.score(features_valid, targets_valid)
print(result)

0.7045101088646968


Доля правильных ответов получилась 0.70, что меньше чем 0.75

Метод **Ближайших соседей**

Определим по валидационной выборке с какими параметрами получается наилучший результат с долей правильных ответов. Проверять будем гиперпараметры leaf_size и n_neighbors

In [9]:
best_model= None
best_result=0
for count in range (1,10):
    for leaf in range(1,50):
        kn = KNeighborsClassifier(leaf_size=leaf, n_neighbors=count)
        kn.fit(features_train, targets_train)
        result = kn.score(features_valid, targets_valid)
        if result>best_result:
            best_model= kn
            best_result= result
print(best_model)
print(best_result)

KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='uniform')
0.7729393468118196


С leaf_size=1 и n_neighbors=8 получается доля правильных ответов 0.77, что больше чем 0.75

## Проверка модели на тестовой выборке

Проверим модели на тестовой выборке. Посмотрим какая получается доля правильных ответов и какое время уходит у каждой модели на прогноз

In [10]:
def perf(model):            # Функция для определения времени, которое модель тратит на прогноз
    start = time.process_time()
    model=model
    model.predict(features_test)
    elapsed = (time.process_time() - start)
    print ('Время выполнения расчета:', elapsed)

**Решающее дерево**

Проверим модель "Решающее дерево" с глубиной дерева, определенной на валидационной выборке = 5. 

In [11]:
tree = DecisionTreeClassifier(random_state=12345, max_depth=5)
tree.fit(features_test, targets_test)
predictions_test_tree=tree.predict(features_test)
print('Accuracy:', accuracy_score(targets_test, predictions_test_tree))
perf(tree)

Accuracy: 0.8413685847589425
Время выполнения расчета: 0.0021951330000007374


Доля правильных ответов получилась больше чем 0.75.

**Случайный лес**

Проверим модель "Случайный лес" с параметрами, определенными на валидационной выборке: max_depth=8 и n_estimators=10

In [12]:
forest = RandomForestClassifier(random_state=12345, n_estimators=10, max_depth=8)
forest.fit(features_test, targets_test)
predictions_test_forest=forest.predict(features_test)
print('Accuracy:', accuracy_score(targets_test, predictions_test_forest))
perf(forest)

Accuracy: 0.8880248833592534
Время выполнения расчета: 0.004624178000000256


Доля правильных ответов получилась больше чем 0.75.

**Логистическая регрессия**

Проверим модель "Логистическая регрессия" на тестовой выборке.

In [13]:
regress= LogisticRegression()
regress.fit(features_train, targets_train)
result_regress = regress.score(features_test, targets_test)
print('Accuracy:',result_regress)
perf(regress)

Accuracy: 0.7060653188180405
Время выполнения расчета: 0.0013326860000013596


Доля правильных ответов получилась меньше чем 0.75, как и на валидационной выборке.

**Метод ближайших соседей**

Проверим модель "Метод ближайших соседей" с параметрами, определенными на валидационной выборке: leaf_size=1 и n_neighbors=8

In [14]:
neighbors = KNeighborsClassifier(leaf_size=1, n_neighbors=8)
neighbors.fit(features_train, targets_train)
predictions_test_neighbors=neighbors.predict(features_test)
print('Accuracy:', accuracy_score(targets_test, predictions_test_neighbors))
perf(neighbors)

Accuracy: 0.76049766718507
Время выполнения расчета: 0.05374818499999989


Доля правильных ответов получилась больше чем 0.75.

Посмотрим на данные всех моделей в таблице.

In [25]:
data = {'model': ['Решающее дерево', 'Случайный лес', 'Логистическая регрессия', 'Метод ближайших соседей'], 
        'accuracy': [0.8413, 0.8880, 0.706, 0.7682], 'time_perfomance':[0.00219, 0.00462, 0.00133, 0.05374],
         'params':[ 'max_depth=5','max_depth=8 и n_estimators=10',' ', 'leaf_size=1 и n_neighbors=8']}
df_pivot = pd.DataFrame.from_dict(data)
display(df_pivot)

,model,accuracy,time_perfomance,params
0,Решающее дерево,0.8413,0.00219,max_depth=5
1,Случайный лес,0.8880,0.00462,max_depth=8 и n_estimators=10
2,Логистическая регрессия,0.7060,0.00133,
3,Метод ближайших соседей,0.7682,0.05374,leaf_size=1 и n_neighbors=8


**Вывод.** 

3 модели из 4 показали долю правильных ответов выше чем 0.75. Только у модели Логистическая регрессия accuracy ниже 0.75. 

Из 3-х моделей с accuracy выше чем 0.75:
- У модели Случайный лес оказалась самая высокая accuracy- 0.88 и средняя скорость расчета. 
- Также достаточно высокая accuracy у модели Решающее дерево - 0.84 и при этом самое быстрое время работы 
- У модели Метод ближайших соседей accuracy ниже чем у моделей Случайный лес и Решающее дерево, и при этом самое высокое время работы.


##  Проверка модели на адекватность

Проверим 3 модели с accuracy больше 0.75, что они не предсказывают одни 0 или 1.

In [16]:
collections.Counter(predictions_test_forest)

Counter({0: 510, 1: 133})

In [17]:
collections.Counter(predictions_test_tree)

Counter({0: 538, 1: 105})

In [18]:
collections.Counter(predictions_test_neighbors)

Counter({0: 568, 1: 75})

Все 3 модели не предсказывают какое то только одно значение.

Проверим качество предсказания  по вектору с константным значением

In [19]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Значений с 0 больше, поэтому константное значение будет 0.

Проверим не тренировочной выборке

In [20]:
predictions_train = pd.Series(targets_train.replace(1, 0))
print(
    f'Качество предсказаний по вектору с константным значением 0:\
    {accuracy_score(targets_train, predictions_train):.2f}'
)

Качество предсказаний по вектору с константным значением 0:    0.69


Проверим на валидационной выборке

In [21]:
predictions_valid = pd.Series(targets_valid.replace(1, 0))
print(
    f'Качество предсказаний по вектору с константным значением 0:\
    {accuracy_score(targets_valid, predictions_valid):.2f}'
)

Качество предсказаний по вектору с константным значением 0:    0.69


Модели показывают более высокую точность. 